In [1]:
%load_ext autoreload

In [2]:
autoreload 2

In [3]:
import os

In [4]:
os.getcwd()

'/Users/mjasiecz/PycharmProjects/new_offer_success_predictor/notebooks'

In [5]:
import numpy as np
import pandas as pd

In [6]:
import sklearn

In [7]:
sklearn.__version__

'0.21.3'

In [8]:
pd.__version__

'0.25.2'

In [9]:
pd.options.display.max_rows = None
pd.options.display.max_columns = None
pd.options.display.max_colwidth = 200

In [10]:
path = '/Users/mjasiecz/Desktop/data_(associate_use_case)/client_database.parquet'

In [11]:
df = pd.read_parquet(path, engine='pyarrow')

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 1308
Data columns (total 15 columns):
offer_class           1289 non-null object
accepted              1289 non-null object
name                  1289 non-null object
gender                1289 non-null object
age                   877 non-null float64
phone_calls           1286 non-null float64
emails                1287 non-null float64
customer_code         1265 non-null object
salary                1281 non-null float64
offer_code            1287 non-null object
customer_type         1287 non-null object
number                1280 non-null object
offer_value           1277 non-null float64
estimated_expenses    1286 non-null float64
center                1306 non-null object
dtypes: float64(6), object(9)
memory usage: 163.6+ KB


In [13]:
from typing import DefaultDict

In [14]:
def initial_exploration(
        data: str or Path or pd.DataFrame,
        sep: str = ',',
        output: bool = True,
        top_values: int = 5,
        desc_stats: bool = True) -> DefaultDict:
    """
    Makes an initial exploration of given CSV file or pandas DataFrame.

    :param data: the path of data table or already create pandas DataFrame
    :param sep: the separator for reading CSV file
    :param output: if set to True, function returns the data
    :param top_values: number of top values, to get from .value_counts()
    :param desc_stats: if set to True, generates descriptive statistics for numeric data
    :return: the dictionary with columns statistics
    """
    df = data if isinstance(data, pd.DataFrame) else read_table(data, sep=sep)
    csv_name = 'DATA' if isinstance(data, pd.DataFrame) else Path(data).stem
    shape = df.shape

    print('\n' + '#' * 200)
    if shape[0] == 0:
        print(f'\n{csv_name} table is empty!!!!\n')
        return None

    data = DefaultDict(list)

    statistics = [
        'all_values_in_column', 'missing_ratio', 'unique_ratio', 'dtype',
        f'top_{top_values}_values', f'top_{top_values}_value_counts'
    ]

    for column in df.columns:
        not_null_count = np.sum(df[column].notnull())
        data[column].append(not_null_count)
        data[column].append((shape[0] - not_null_count) / shape[0])
        data[column].append(df[column].nunique() / not_null_count if not_null_count > 0 else 0)
        data[column].append(df[column].dtype.name)

        values_counts = df[column].value_counts()
        data[column].append(values_counts.index.tolist()[:top_values])
        data[column].append(values_counts.values.tolist()[:top_values])

    if desc_stats:
        statistics += ['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max', 'sum']
        numeric_columns = set(df.select_dtypes(include='number').columns)
        descriptive_stats = df.describe()
        for column in df.columns:
            if column in numeric_columns:
                data[column].extend(descriptive_stats[column].tolist() + [df[column].sum()])
            else:
                data[column].extend(['N/A'] * 9)

    print(f'\nName: {csv_name}')
    print(f'Shape: {shape}')

    with pd.option_context('display.max_rows', None, 'display.max_columns', None,
                           'display.max_colwidth', 200,
                           'display.width', 250, 'display.float_format', '{:4,.2f}'.format):
        print('\n********** COLUMNS STATISTICS **********')
        display(pd.DataFrame.from_dict(data, orient='index', columns=statistics))

    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        print('\n********** SAMPLE FROM DATA **********')
        display(df.head(3))
        print()

    return data if output else None

In [15]:
initial_exploration(df, top_values=10, desc_stats=False);


########################################################################################################################################################################################################

Name: DATA
Shape: (1309, 15)

********** COLUMNS STATISTICS **********


,all_values_in_column,missing_ratio,unique_ratio,dtype,top_10_values,top_10_value_counts
offer_class,1289,0.02,0.00,object,"[Premium, Medium, High]","[695, 322, 272]"
accepted,1289,0.02,0.00,object,"[no, yes]","[800, 489]"
name,1289,0.02,1.00,object,"[3B9737D697A449E, 891F4C60A617408, D39B668BB99642E, BA3BDAF438664FA, 32A600E1F9614FD, A2CC78E3D3B3407, E6F8E33BCC69404, 90B8227DE1294B2, 5EFAE0E0E48D4FB, 404EB955636A46B]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
gender,1289,0.02,0.00,object,"[male, female]","[831, 458]"
age,877,0.33,0.08,float64,"[24.0, 21.0, 22.0, 30.0, 18.0, 25.0, 28.0, 36.0, 26.0, 27.0]","[46, 41, 41, 40, 38, 34, 32, 30, 30, 30]"
phone_calls,1286,0.02,0.01,float64,"[0.0, 1.0, 2.0, 4.0, 3.0, 8.0, 5.0]","[877, 311, 42, 22, 20, 9, 5]"
emails,1287,0.02,0.01,float64,"[1.0, 0.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 9.0, 10.0]","[569, 503, 132, 60, 10, 5, 4, 2, 1, 1]"
customer_code,1265,0.03,0.71,object,"[CA. 2343, PC 17608, 3101295, S.O.C. 14879, 347082, 1601, CA 2144, 347077, 19950, 382652]","[11, 7, 7, 7, 7, 7, 7, 7, 6, 6]"
salary,1281,0.02,0.22,float64,"[1300.0, 805.0, 775.0, 2600.0, 789.58, 1050.0, 777.5, 722.92, 792.5, 2655.0]","[59, 57, 53, 48, 47, 35, 26, 24, 23, 22]"
offer_code,1287,0.02,0.86,object,"[0.00E+00, 85, 773, 12, 1D7, E09, 9EF, 943, E57, CFD]","[5, 4, 4, 3, 3, 3, 3, 3, 3, 3]"



********** SAMPLE FROM DATA **********


,offer_class,accepted,name,gender,age,phone_calls,emails,customer_code,salary,offer_code,customer_type,number,offer_value,estimated_expenses,center
0,Medium,yes,C7CBB5C5613449B,female,29.0,0.0,0.0,24160,21133.75,4AB,S,9E9FA,57.426571,4692.0,A
1,Medium,yes,CFD09C0248BB417,male,NaN,1.0,2.0,113781,15155.00,61A,S,1E53D,141.639912,3164.0,A
2,Medium,no,A2A0DC541977473,female,NaN,1.0,3.0,113781,15155.00,DB4,S,1.36E+06,154.821130,1852.0,A


offer_value is unique - find pattern to calculate it!

In [16]:
# age has a lot of nan values!

In [17]:
df[df['number'] == 'C8A9B']

,offer_class,accepted,name,gender,age,phone_calls,emails,customer_code,salary,offer_code,customer_type,number,offer_value,estimated_expenses,center
1076,Premium,no,3BF3D98A66FA40B,male,NaN,3.0,1.0,349909,2107.50,205,S,C8A9B,88.040275,2204.0,A
1166,Premium,no,19127F3199944C4,male,NaN,2.0,1.0,2662,2167.92,149,C,C8A9B,101.863080,2190.0,B


In [18]:
df[df['age'] < 18]

,offer_class,accepted,name,gender,age,phone_calls,emails,customer_code,salary,offer_code,customer_type,number,offer_value,estimated_expenses,center


In [19]:
# drop full na rows

In [20]:
df[df['name'].isna()]

,offer_class,accepted,name,gender,age,phone_calls,emails,customer_code,salary,offer_code,customer_type,number,offer_value,estimated_expenses,center
1289,None,None,None,None,NaN,NaN,NaN,None,NaN,None,None,None,NaN,NaN,A
1290,None,None,None,None,NaN,NaN,NaN,None,NaN,None,None,None,NaN,NaN,A
1291,None,None,None,None,NaN,NaN,NaN,None,NaN,None,None,None,NaN,NaN,A
1292,None,None,None,None,NaN,NaN,NaN,None,NaN,None,None,None,NaN,NaN,B
1293,None,None,None,None,NaN,NaN,NaN,None,NaN,None,None,None,NaN,NaN,A
1294,None,None,None,None,NaN,NaN,NaN,None,NaN,None,None,None,NaN,NaN,B
1295,None,None,None,None,NaN,NaN,NaN,None,NaN,None,None,None,NaN,NaN,B
1296,None,None,None,None,NaN,NaN,NaN,None,NaN,None,None,None,NaN,NaN,B
1297,None,None,None,None,NaN,NaN,NaN,None,NaN,None,None,None,NaN,NaN,A
1298,None,None,None,None,NaN,NaN,NaN,None,NaN,None,None,None,NaN,NaN,A


In [21]:
df[['customer_type', 'accepted', 'offer_class']].groupby(['customer_type', 'accepted']).count()

offer_class
customer_type accepted             
C             no                118
              yes               150
Q             no                 78
              yes                42
S             no                604
              yes               295

In [22]:
df[df['customer_code'] == 'CA. 2343']

,offer_class,accepted,name,gender,age,phone_calls,emails,customer_code,salary,offer_code,customer_type,number,offer_value,estimated_expenses,center
1150,Premium,no,7AF493A2CD8E4FA,male,NaN,8.0,3.0,CA. 2343,6955.0,6EC,S,12816,99.692669,4688.0,A
1151,Premium,no,3A9C52F8A32046F,male,NaN,8.0,3.0,CA. 2343,6955.0,632,S,57AE7,54.139760,2401.0,A
1152,Premium,no,881815EF2A8A484,female,NaN,8.0,2.0,CA. 2343,6955.0,6DB,S,89CAA,54.440937,7469.0,B
1153,Premium,no,B551CD3E07904F1,female,NaN,8.0,3.0,CA. 2343,6955.0,75C,S,29DF0,82.027279,6816.0,B
1154,Premium,no,BDF502F66E634B1,female,NaN,8.0,2.0,CA. 2343,6955.0,90C,S,F7560,178.518323,3922.0,B
1155,Premium,no,9044BC863E934F0,female,NaN,8.0,2.0,CA. 2343,6955.0,33E,S,A0939,228.625174,2829.0,B
1156,Premium,no,B02195C26B024EC,male,NaN,8.0,2.0,CA. 2343,6955.0,F51,S,E013D,85.970912,3691.0,B
1157,Premium,no,3FCFB49CFC814CA,male,NaN,8.0,2.0,CA. 2343,6955.0,B87,S,6E0BF,118.113638,4550.0,A
1158,Premium,no,46D214F8C0684A2,male,NaN,8.0,2.0,CA. 2343,6955.0,DE9,S,39A6A,138.979042,7057.0,B
1159,Premium,no,B1F4013EEA474A2,male,NaN,1.0,10.0,CA. 2343,6955.0,03C,S,8A64D,50.962574,2048.0,B


In [23]:
df[df['customer_code'] == '1601']

,offer_class,accepted,name,gender,age,phone_calls,emails,customer_code,salary,offer_code,customer_type,number,offer_value,estimated_expenses,center
667,Premium,yes,DE3C3F57CD404EC,male,32.0,0.0,1.0,1601,5649.58,16A,S,C07B5,94.725194,4611.0,B
793,Premium,yes,8535BBCA690A4AE,male,NaN,0.0,1.0,1601,5649.58,6FF,S,42EFB,121.371033,2106.0,B
844,Premium,yes,385BDA8155A5457,male,NaN,0.0,0.0,1601,5649.58,15A,S,D21BC,123.100625,7199.0,B
929,Premium,yes,2D0F5F099E5C48C,male,NaN,0.0,1.0,1601,5649.58,CCB,S,7E88C,144.808531,4491.0,A
930,Premium,no,AF47DD3618B44D0,male,NaN,0.0,0.0,1601,5649.58,42E,S,68DC3,84.914862,6907.0,B
932,Premium,yes,C73C35EA60D54E1,male,26.0,0.0,0.0,1601,5649.58,330,S,6D8FC,79.856124,7137.0,A
955,Premium,no,1022A3C930F94BB,male,28.0,0.0,1.0,1601,5649.58,74E,S,B499E,103.704159,7035.0,B


In [24]:
df[df['customer_code'] == 'CA 2144']

,offer_class,accepted,name,gender,age,phone_calls,emails,customer_code,salary,offer_code,customer_type,number,offer_value,estimated_expenses,center
812,Premium,no,FC69C7A64DE0467,male,NaN,5.0,3.0,CA 2144,4690.0,D44,S,B1663,185.665977,5819.0,B
813,Premium,no,877132CA4012418,male,NaN,5.0,2.0,CA 2144,4690.0,7DD,S,2E742,91.641133,4996.0,B
814,Premium,no,CED2AD95912D469,female,NaN,5.0,3.0,CA 2144,4690.0,49C,S,A6A13,160.726964,3824.0,B
815,Premium,no,14E7265E6DDB42E,female,NaN,5.0,3.0,CA 2144,4690.0,603,S,A9927,169.667561,5813.0,A
816,Premium,no,D43760FE0E274DB,male,NaN,5.0,2.0,CA 2144,4690.0,3C0,S,E41BF,169.000495,5427.0,A
817,Premium,no,204D35504485488,male,40.0,1.0,7.0,CA 2144,4690.0,C6E,S,68236,227.101595,6667.0,A
818,Premium,no,717E9A394E494C9,female,43.0,1.0,7.0,CA 2144,4690.0,F87,S,D7347,119.575502,5491.0,A


In [25]:
df['customer_code'].nunique()

899

In [26]:
df[~df['customer_code'].duplicated(keep=False)]['customer_code'].nunique()

689

In [27]:
duplicated_customer_code = df[df['customer_code'].duplicated(keep=False)]

In [28]:
duplicated_customer_code

,offer_class,accepted,name,gender,age,phone_calls,emails,customer_code,salary,offer_code,customer_type,number,offer_value,estimated_expenses,center
0,Medium,yes,C7CBB5C5613449B,female,29.0,0.0,0.0,24160,21133.75,4AB,S,9E9FA,57.426571,4692.0,A
1,Medium,yes,CFD09C0248BB417,male,NaN,1.0,2.0,113781,15155.00,61A,S,1E53D,141.639912,3164.0,A
2,Medium,no,A2A0DC541977473,female,NaN,1.0,3.0,113781,15155.00,DB4,S,1.36E+06,154.821130,1852.0,A
3,Medium,no,9068458EB70D427,male,30.0,1.0,2.0,113781,15155.00,9B6,S,F6529,106.256196,3753.0,B
4,Medium,no,46F0CD19CF71429,female,25.0,1.0,2.0,113781,15155.00,191,S,E2FDF,139.237147,2410.0,A
6,Medium,yes,A472C30155B64A4,female,63.0,1.0,1.0,13502,7795.83,801,S,1B8E6,60.742667,4089.0,A
8,Medium,yes,F1DB146F2D264B9,female,53.0,2.0,1.0,11769,5147.92,D8C,S,4E905,75.369051,4828.0,A
10,Medium,no,F36F20E6C4C9460,male,47.0,1.0,0.0,PC 17757,22752.50,2,C,C7C42,128.277347,3126.0,A
11,Medium,yes,B6409DE9C35C4BB,female,18.0,1.0,0.0,PC 17757,22752.50,3F2,C,30B1E,161.352318,4619.0,A
12,Medium,yes,1B1B5AFE5EF249A,female,24.0,0.0,1.0,PC 17477,6930.00,0FD,C,E6D29,87.760293,7630.0,B


In [29]:
np.mean(df[df['salary'] < np.mean(df['salary'])]['age'])

31.689922480620154

In [30]:
np.mean(df[df['salary'] > np.mean(df['salary'])]['age'])

38.46

In [31]:
np.mean(df[df['gender'] == 'female']['salary'])

4555.110901098901

In [32]:
np.mean(df[df['gender'] == 'male']['salary'])

2604.431876513317

In [33]:
# do i know anything about new offer?

In [34]:
# investigate correlation between salary and customer_code <- ANOVA (check assumptions)

In [35]:
# we can fill in missing salary using customer_code

In [36]:
# check correlation between age numeric cols (i.e. age and salary) (gender and salary)

In [37]:
# new offer can be based on offer_class/ offer_value

In [38]:
# customer_code 7534 has different salaries

In [39]:
duplicated_customer_code.sort_values(by = 'customer_code').head(600)

,offer_class,accepted,name,gender,age,phone_calls,emails,customer_code,salary,offer_code,customer_type,number,offer_value,estimated_expenses,center
194,Medium,yes,35FAF028D859473,female,NaN,0.0,0.0,110152,8650.00,83,S,2EF1C,53.053735,5512.0,A
244,Medium,yes,0EF1290864DB478,female,33.0,0.0,1.0,110152,8650.00,2DF,S,37CAC,145.050597,3841.0,B
66,Medium,yes,3EB0BF094F65404,female,30.0,0.0,1.0,110152,8650.00,6B3,S,67810,101.338875,5286.0,B
290,Medium,yes,195376F057454E8,female,39.0,1.0,2.0,110413,7965.00,D46,S,AA2D0,86.057107,5917.0,B
289,Medium,no,D072215BC075451,male,52.0,1.0,2.0,110413,7965.00,698,S,A8C58,184.487056,5363.0,B
288,Medium,yes,E5EDAE9C1F3A439,female,18.0,0.0,2.0,110413,7965.00,EB8,S,A8F53,101.528534,6357.0,B
73,Medium,no,D17218C54DC9435,male,NaN,0.0,1.0,110465,5200.00,805,S,4A7E5,71.934256,2655.0,A
231,Medium,no,B17D2FF7DDD9417,male,47.0,0.0,0.0,110465,5200.00,CAF,S,00F96,203.058674,3326.0,B
302,Medium,no,B73AA87FC880490,male,64.0,1.0,0.0,110813,7525.00,CB1,C,9438A,74.252057,5537.0,B
303,Medium,yes,D1F0131EBAD9424,female,60.0,1.0,0.0,110813,7525.00,D78,C,E7328,125.563875,2237.0,B


In [40]:
df_name = df[['name', 'accepted']].dropna(subset=['name'])

In [41]:
df_name[df_name['name'].str.startswith('A')]['accepted'].value_counts()

no     45
yes    40
Name: accepted, dtype: int64

In [42]:
df_name[df_name['name'].str.startswith('B')]['accepted'].value_counts()

no     53
yes    30
Name: accepted, dtype: int64

In [43]:
df_name[df_name['name'].str.startswith('C')]['accepted'].value_counts()

no     48
yes    30
Name: accepted, dtype: int64

In [44]:
df_name[df_name['name'].str.startswith('D')]['accepted'].value_counts()

no     55
yes    37
Name: accepted, dtype: int64

In [45]:
df_name[df_name['name'].str.startswith('E')]['accepted'].value_counts()

no     47
yes    36
Name: accepted, dtype: int64

In [46]:
df_name[df_name['name'].str.startswith('F')]['accepted'].value_counts()

no     55
yes    28
Name: accepted, dtype: int64

In [47]:
df_name[df_name['name'].str.startswith('G')]['accepted'].value_counts()

Series([], Name: accepted, dtype: int64)

In [48]:
np.sum(df_name['name'].str.len() != 15)

3

In [49]:
df_name[df_name['name'].str.slice(0,1).str.isdigit()]['accepted'].value_counts()

no     497
yes    288
Name: accepted, dtype: int64

In [50]:
# reduce memory before modelling? to speed it up?

In [51]:
df['offer_code'].dtype

dtype('O')

In [52]:
df['age'].dtype

dtype('float64')

In [53]:
# pilnowac aby nie przetrenowac / nie niedotrenowac danych (no overfitting/underfitting)
# zastanowic sie, jak reprezentatywne bedzie to do nowych danych

In [54]:
df.dropna(subset=['offer_code'])[df['offer_code'].dropna().str.contains('E\+')]['accepted'].value_counts()

no     28
yes     5
Name: accepted, dtype: int64

In [55]:
np.sum(df['customer_code'].isna())

44

In [56]:
df1 = df.copy(deep=True)

In [57]:
df1.dropna(subset = ['customer_code'], inplace=True)

In [58]:
df1 = df1.reset_index()

In [59]:
df1.head()

,index,offer_class,accepted,name,gender,age,phone_calls,emails,customer_code,salary,offer_code,customer_type,number,offer_value,estimated_expenses,center
0,0,Medium,yes,C7CBB5C5613449B,female,29.0,0.0,0.0,24160,21133.75,4AB,S,9E9FA,57.426571,4692.0,A
1,1,Medium,yes,CFD09C0248BB417,male,NaN,1.0,2.0,113781,15155.00,61A,S,1E53D,141.639912,3164.0,A
2,2,Medium,no,A2A0DC541977473,female,NaN,1.0,3.0,113781,15155.00,DB4,S,1.36E+06,154.821130,1852.0,A
3,3,Medium,no,9068458EB70D427,male,30.0,1.0,2.0,113781,15155.00,9B6,S,F6529,106.256196,3753.0,B
4,4,Medium,no,46F0CD19CF71429,female,25.0,1.0,2.0,113781,15155.00,191,S,E2FDF,139.237147,2410.0,A


In [60]:
df2 = df.dropna(subset=['number'])

In [61]:
df2[df2['number'].str.contains('E\+')]['accepted'].value_counts()

no     23
yes    14
Name: accepted, dtype: int64

In [62]:
df[['offer_class', 'customer_type', 'accepted']].groupby(['customer_type', 'accepted']).count()

offer_class
customer_type accepted             
C             no                118
              yes               150
Q             no                 78
              yes                42
S             no                604
              yes               295

In [63]:
df[['offer_class', 'center', 'accepted']].groupby(['center', 'accepted']).count()

offer_class
center accepted             
A      no                369
       yes               220
B      no                429
       yes               268

In [64]:
df1[df1['customer_code'].str.contains('[A-Za-z]')]['accepted'].value_counts()

no     199
yes    129
Name: accepted, dtype: int64

In [65]:
df1[~df1['customer_code'].str.contains('[A-Za-z]')]['accepted'].value_counts()

no     586
yes    351
Name: accepted, dtype: int64

In [66]:
df1.drop(columns = ['index'], inplace=True)

In [67]:
np.sum(df1['customer_code'].isna())

0

In [68]:
df1[df1['customer_code'].str.contains('PC')]['accepted'].value_counts()

yes    58
no     33
Name: accepted, dtype: int64

In [69]:
df1[df1['customer_code'].str.contains('CA')]['accepted'].value_counts()

no     19
yes     2
Name: accepted, dtype: int64

In [70]:
df1[df1['customer_code'].str.contains('S\.O\.C\.')]['accepted'].value_counts()

no    7
Name: accepted, dtype: int64

In [71]:
df1[df1['customer_code'].str.contains('F.C')]['accepted'].value_counts()

yes    6
no     5
Name: accepted, dtype: int64

In [72]:
df1[df1['customer_code'].str.contains('SOTON')]['accepted'].value_counts()

no     15
yes     4
Name: accepted, dtype: int64

In [73]:
df1[df1['customer_code'].str.contains('STON')]['accepted'].value_counts()

no     12
yes     7
Name: accepted, dtype: int64

In [74]:
df1[df1['customer_code'].str.contains('SC')]['accepted'].value_counts()

no     13
yes    12
Name: accepted, dtype: int64

In [75]:
df1[df1['customer_code'].str.contains('PARIS')]['accepted'].value_counts()

yes    8
no     6
Name: accepted, dtype: int64

In [76]:
df1[df1['customer_code'].str.contains('A')]['accepted'].value_counts()

no     91
yes    33
Name: accepted, dtype: int64

In [77]:
unique_customer_code = df[~df['customer_code'].duplicated(keep=False)]

In [78]:
duplicated_customer_code['accepted'].value_counts()

yes    304
no     296
Name: accepted, dtype: int64

In [79]:
unique_customer_code['accepted'].value_counts()

no     504
yes    185
Name: accepted, dtype: int64

In [80]:
df[['customer_code', 'salary']].groupby('customer_code').agg('mean')

,salary
customer_code,
110152,8650.000
110413,7965.000
110465,5200.000
110469,2600.000
110489,2655.000
110564,2655.000
110813,7525.000
111163,2600.000
111240,3350.000


In [81]:
# customer_code to drop, (but i need to check it in formal way) it probably contains same information as salary

do data is duplicated?

In [82]:
numeric_columns = ['age', 'phone_calls', 'emails', 'salary', 'offer_value', 'estimated_expenses']

In [83]:
df[numeric_columns].describe()

,age,phone_calls,emails,salary,offer_value,estimated_expenses
count,877.000000,1286.000000,1287.000000,1281.000000,1277.000000,1286.000000
mean,33.511973,0.497667,0.881896,3297.296011,128.693732,4576.947900
std,12.247058,1.041360,1.022320,5034.240427,57.677807,1909.458459
min,18.000000,0.000000,0.000000,0.000000,50.022619,1257.000000
25%,24.000000,0.000000,0.000000,789.580000,82.632085,2934.750000
50%,30.000000,0.000000,1.000000,1445.420000,118.645478,4544.000000
75%,41.000000,1.000000,1.000000,3127.500000,162.482961,6225.750000
max,80.000000,8.000000,10.000000,51232.920000,368.668534,7891.000000


In [84]:
# look at outliers i.e. client with salary 51232...

In [85]:
df[numeric_columns].quantile(0.90)

age                     51.000000
phone_calls              1.000000
emails                   2.000000
salary                7795.830000
offer_value            210.274666
estimated_expenses    7241.000000
Name: 0.9, dtype: float64

In [86]:
df[numeric_columns].quantile(0.95)

age                      58.000000
phone_calls               2.000000
emails                    3.000000
salary                13365.000000
offer_value             241.682618
estimated_expenses     7563.500000
Name: 0.95, dtype: float64

In [87]:
df[df['salary'] >= 13365.000000]['accepted'].value_counts()

yes    45
no     20
Name: accepted, dtype: int64

In [88]:
df[df['salary'] < 789.580000]['accepted'].value_counts()

no     211
yes     72
Name: accepted, dtype: int64

In [89]:
# zbalansowac zbior danych - zrobic sampling (SMOTE lub cos z nearest neighbors - patrz fraud detection) ale tylko na danych treningowych (zbioru testowego nie tykać!), 30-40% danych na zbior testowy, missing_values, outliers, EDA

In [90]:
# 2 testowe -> aby uniknac przetrenowania. testowac wyniki na pierwszym i tam robic tuning modelu,
# ale koncowy model wybrac na podstawie najmniejszej roznicy w performance miedzy dwoma zbiorami/ lub na podstawie wyniku na 2 zbiorze -> lub zrobic jakich miks tych 2 wymagan

In [91]:
# moze jakies testy statystyczne?

In [92]:
# do some benchmarks on test set

In [93]:
# all no, all yes, based only on numeric columns, no dimensionality reduction

In [94]:
# use autoencoder to reduce dimensionality

In [95]:
# what metrics? (wybrać 3-4 metryki :) )()

In [96]:
# what with people with 0 salary?

In [97]:
# difference between salary and estimated_expenses

In [98]:
# encode class_offer to three variables

In [99]:
# .predict_proba

In [100]:
df['difference_between_salary_and_estimated_expenses'] = df['salary']-df['estimated_expenses']

In [101]:
df[df['difference_between_salary_and_estimated_expenses'] < 0]['accepted'].value_counts()

no     678
yes    322
Name: accepted, dtype: int64

In [102]:
df[df['difference_between_salary_and_estimated_expenses'] > 0]['accepted'].value_counts()

yes    159
no     119
Name: accepted, dtype: int64

In [103]:
# remember estimated! (it is just estimation, not real data <- ask PO) if these are customer expenses?

In [104]:
df.head()

,offer_class,accepted,name,gender,age,phone_calls,emails,customer_code,salary,offer_code,customer_type,number,offer_value,estimated_expenses,center,difference_between_salary_and_estimated_expenses
0,Medium,yes,C7CBB5C5613449B,female,29.0,0.0,0.0,24160,21133.75,4AB,S,9E9FA,57.426571,4692.0,A,16441.75
1,Medium,yes,CFD09C0248BB417,male,NaN,1.0,2.0,113781,15155.00,61A,S,1E53D,141.639912,3164.0,A,11991.00
2,Medium,no,A2A0DC541977473,female,NaN,1.0,3.0,113781,15155.00,DB4,S,1.36E+06,154.821130,1852.0,A,13303.00
3,Medium,no,9068458EB70D427,male,30.0,1.0,2.0,113781,15155.00,9B6,S,F6529,106.256196,3753.0,B,11402.00
4,Medium,no,46F0CD19CF71429,female,25.0,1.0,2.0,113781,15155.00,191,S,E2FDF,139.237147,2410.0,A,12745.00


In [105]:
from sklearn.model_selection import train_test_split

In [106]:
benchmark_df = df[['offer_class', 'accepted', 'name', 'gender', 'age', 'phone_calls', 'emails', 'salary', 'offer_value', 'estimated_expenses']]

In [107]:
benchmark_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 1308
Data columns (total 10 columns):
offer_class           1289 non-null object
accepted              1289 non-null object
name                  1289 non-null object
gender                1289 non-null object
age                   877 non-null float64
phone_calls           1286 non-null float64
emails                1287 non-null float64
salary                1281 non-null float64
offer_value           1277 non-null float64
estimated_expenses    1286 non-null float64
dtypes: float64(6), object(4)
memory usage: 112.5+ KB


In [108]:
benchmark_df = benchmark_df.dropna(subset = ['accepted'])

In [109]:
benchmark_df.set_index('name', inplace=True)

In [110]:
benchmark_df.head()

,offer_class,accepted,gender,age,phone_calls,emails,salary,offer_value,estimated_expenses
name,,,,,,,,,
C7CBB5C5613449B,Medium,yes,female,29.0,0.0,0.0,21133.75,57.426571,4692.0
CFD09C0248BB417,Medium,yes,male,NaN,1.0,2.0,15155.00,141.639912,3164.0
A2A0DC541977473,Medium,no,female,NaN,1.0,3.0,15155.00,154.821130,1852.0
9068458EB70D427,Medium,no,male,30.0,1.0,2.0,15155.00,106.256196,3753.0
46F0CD19CF71429,Medium,no,female,25.0,1.0,2.0,15155.00,139.237147,2410.0


In [111]:
benchmark_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1289 entries, C7CBB5C5613449B to B8A3C4B5FDE74D2
Data columns (total 9 columns):
offer_class           1289 non-null object
accepted              1289 non-null object
gender                1289 non-null object
age                   877 non-null float64
phone_calls           1286 non-null float64
emails                1287 non-null float64
salary                1281 non-null float64
offer_value           1277 non-null float64
estimated_expenses    1286 non-null float64
dtypes: float64(6), object(3)
memory usage: 100.7+ KB


In [112]:
X = benchmark_df.copy(deep=True)

In [113]:
X['accepted'].value_counts()

no     800
yes    489
Name: accepted, dtype: int64

In [114]:
X.head()

,offer_class,accepted,gender,age,phone_calls,emails,salary,offer_value,estimated_expenses
name,,,,,,,,,
C7CBB5C5613449B,Medium,yes,female,29.0,0.0,0.0,21133.75,57.426571,4692.0
CFD09C0248BB417,Medium,yes,male,NaN,1.0,2.0,15155.00,141.639912,3164.0
A2A0DC541977473,Medium,no,female,NaN,1.0,3.0,15155.00,154.821130,1852.0
9068458EB70D427,Medium,no,male,30.0,1.0,2.0,15155.00,106.256196,3753.0
46F0CD19CF71429,Medium,no,female,25.0,1.0,2.0,15155.00,139.237147,2410.0


In [115]:
# add some unit testing to the code

In [116]:
X['is_male'] = X['gender'] == 'male'

In [117]:
X['is_male'].value_counts()

True     831
False    458
Name: is_male, dtype: int64

In [118]:
# czy chcmy dac mozliwosc klientowi aby sprawdzal, dla oferty ktorej klasy ma najwieksze prawdopodobienstwo, zaakceptowania tej oferty dla danego klienta?

In [119]:
X['medium_offer'] = X['offer_class'] == 'Medium'

In [120]:
X['premium_offer'] = X['offer_class'] == 'Premium'

In [121]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1289 entries, C7CBB5C5613449B to B8A3C4B5FDE74D2
Data columns (total 12 columns):
offer_class           1289 non-null object
accepted              1289 non-null object
gender                1289 non-null object
age                   877 non-null float64
phone_calls           1286 non-null float64
emails                1287 non-null float64
salary                1281 non-null float64
offer_value           1277 non-null float64
estimated_expenses    1286 non-null float64
is_male               1289 non-null bool
medium_offer          1289 non-null bool
premium_offer         1289 non-null bool
dtypes: bool(3), float64(6), object(3)
memory usage: 104.5+ KB


In [122]:
X.head()

,offer_class,accepted,gender,age,phone_calls,emails,salary,offer_value,estimated_expenses,is_male,medium_offer,premium_offer
name,,,,,,,,,,,,
C7CBB5C5613449B,Medium,yes,female,29.0,0.0,0.0,21133.75,57.426571,4692.0,False,True,False
CFD09C0248BB417,Medium,yes,male,NaN,1.0,2.0,15155.00,141.639912,3164.0,True,True,False
A2A0DC541977473,Medium,no,female,NaN,1.0,3.0,15155.00,154.821130,1852.0,False,True,False
9068458EB70D427,Medium,no,male,30.0,1.0,2.0,15155.00,106.256196,3753.0,True,True,False
46F0CD19CF71429,Medium,no,female,25.0,1.0,2.0,15155.00,139.237147,2410.0,False,True,False


In [123]:
X.drop(columns=['offer_class', 'gender'], inplace=True)

In [124]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1289 entries, C7CBB5C5613449B to B8A3C4B5FDE74D2
Data columns (total 10 columns):
accepted              1289 non-null object
age                   877 non-null float64
phone_calls           1286 non-null float64
emails                1287 non-null float64
salary                1281 non-null float64
offer_value           1277 non-null float64
estimated_expenses    1286 non-null float64
is_male               1289 non-null bool
medium_offer          1289 non-null bool
premium_offer         1289 non-null bool
dtypes: bool(3), float64(6), object(1)
memory usage: 84.3+ KB


In [125]:
X[X['is_male']]['age'].mean()

33.65520282186949

In [126]:
X[~X['is_male']]['age'].mean()

33.25

In [127]:
X[X['premium_offer']]['offer_value'].mean()

127.56360901743813

In [128]:
X[X['medium_offer']]['offer_value'].mean()

129.6396043689028

In [129]:
other_offer = X[~X['medium_offer']]

In [130]:
other_offer = other_offer[~other_offer['premium_offer']]

In [131]:
other_offer['offer_value'].mean()

130.44524859708488

In [132]:
X['offer_value'].mean()

128.69373183905245

In [133]:
X.loc[X['offer_value'].isna(), 'offer_value'] = X['offer_value'].mean()

In [134]:
X.head()

,accepted,age,phone_calls,emails,salary,offer_value,estimated_expenses,is_male,medium_offer,premium_offer
name,,,,,,,,,,
C7CBB5C5613449B,yes,29.0,0.0,0.0,21133.75,57.426571,4692.0,False,True,False
CFD09C0248BB417,yes,NaN,1.0,2.0,15155.00,141.639912,3164.0,True,True,False
A2A0DC541977473,no,NaN,1.0,3.0,15155.00,154.821130,1852.0,False,True,False
9068458EB70D427,no,30.0,1.0,2.0,15155.00,106.256196,3753.0,True,True,False
46F0CD19CF71429,no,25.0,1.0,2.0,15155.00,139.237147,2410.0,False,True,False


In [135]:
X['accepted'].value_counts()

no     800
yes    489
Name: accepted, dtype: int64

In [136]:
X.loc[X['age'].isna(), 'age'] = X['age'].mean()

In [137]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1289 entries, C7CBB5C5613449B to B8A3C4B5FDE74D2
Data columns (total 10 columns):
accepted              1289 non-null object
age                   1289 non-null float64
phone_calls           1286 non-null float64
emails                1287 non-null float64
salary                1281 non-null float64
offer_value           1289 non-null float64
estimated_expenses    1286 non-null float64
is_male               1289 non-null bool
medium_offer          1289 non-null bool
premium_offer         1289 non-null bool
dtypes: bool(3), float64(6), object(1)
memory usage: 84.3+ KB


In [138]:
X.dropna(subset = ['phone_calls'], inplace=True)

In [139]:
X.dropna(subset = ['emails'], inplace=True)

In [140]:
X.dropna(subset = ['salary'], inplace=True)

In [141]:
X.dropna(subset = ['estimated_expenses'], inplace=True)

In [142]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1273 entries, C7CBB5C5613449B to B8A3C4B5FDE74D2
Data columns (total 10 columns):
accepted              1273 non-null object
age                   1273 non-null float64
phone_calls           1273 non-null float64
emails                1273 non-null float64
salary                1273 non-null float64
offer_value           1273 non-null float64
estimated_expenses    1273 non-null float64
is_male               1273 non-null bool
medium_offer          1273 non-null bool
premium_offer         1273 non-null bool
dtypes: bool(3), float64(6), object(1)
memory usage: 83.3+ KB


In [143]:
X['target'] = X['accepted'] == 'yes'

In [144]:
y = X['target']

In [145]:
# y = X['accepted']

In [146]:
X = X.drop(columns = ['accepted', 'target'])

In [147]:
y.head()

name
C7CBB5C5613449B     True
CFD09C0248BB417     True
A2A0DC541977473    False
9068458EB70D427    False
46F0CD19CF71429    False
Name: target, dtype: bool

In [148]:
y.shape

(1273,)

In [149]:
X.head()

,age,phone_calls,emails,salary,offer_value,estimated_expenses,is_male,medium_offer,premium_offer
name,,,,,,,,,
C7CBB5C5613449B,29.000000,0.0,0.0,21133.75,57.426571,4692.0,False,True,False
CFD09C0248BB417,33.511973,1.0,2.0,15155.00,141.639912,3164.0,True,True,False
A2A0DC541977473,33.511973,1.0,3.0,15155.00,154.821130,1852.0,False,True,False
9068458EB70D427,30.000000,1.0,2.0,15155.00,106.256196,3753.0,True,True,False
46F0CD19CF71429,25.000000,1.0,2.0,15155.00,139.237147,2410.0,False,True,False


In [150]:
y.head()

name
C7CBB5C5613449B     True
CFD09C0248BB417     True
A2A0DC541977473    False
9068458EB70D427    False
46F0CD19CF71429    False
Name: target, dtype: bool

In [151]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=44, test_size=0.3)

In [152]:
X_train.head()

,age,phone_calls,emails,salary,offer_value,estimated_expenses,is_male,medium_offer,premium_offer
name,,,,,,,,,
3ADEFBB646EE47D,39.0,0.0,6.0,2912.50,69.413718,5716.0,False,False,True
951A4A2911CD42A,62.0,0.0,1.0,968.75,213.988314,3229.0,True,False,False
97BCE56439624A7,53.0,0.0,0.0,2850.00,110.470351,7244.0,True,True,False
C275791942724E6,48.0,1.0,1.0,5200.00,128.873564,6663.0,True,True,False
8B761D253C204DD,36.0,0.0,4.0,3900.00,129.279096,4958.0,False,False,False


In [153]:
X_test.head()

,age,phone_calls,emails,salary,offer_value,estimated_expenses,is_male,medium_offer,premium_offer
name,,,,,,,,,
FA9B6C2B73484DB,33.511973,0.0,1.0,851.67,130.948345,2700.0,True,False,True
3F05B4A0E6B044A,32.000000,0.0,1.0,789.58,220.420406,2417.0,True,False,True
E53A74A9B6F94F6,30.000000,0.0,1.0,9350.00,123.110690,4926.0,False,True,False
613DF0DAE57441A,33.511973,0.0,0.0,775.00,105.050989,2999.0,False,False,True
3C15CA99C3E7446,33.511973,1.0,1.0,5200.00,102.158908,5711.0,False,True,False


In [154]:
# help(train_test_split)

In [155]:
y_train.value_counts()

False    546
True     345
Name: target, dtype: int64

In [156]:
551/340

1.6205882352941177

In [157]:
y_test.value_counts()

False    247
True     135
Name: target, dtype: int64

In [158]:
242/140

1.7285714285714286

In [159]:
247+135

382

In [160]:
247/382 #oznaczono wszystkich jako no

0.6465968586387435

In [161]:
135/382 #oznaczono wszystkich jako yes

0.35340314136125656

In [162]:
from sklearn.neighbors import KNeighborsClassifier

In [163]:
y_train.value_counts()

False    546
True     345
Name: target, dtype: int64

In [164]:
y_train.shape[0]+y_test.shape[0]

1273

In [165]:
import numpy as np

In [166]:
clf = KNeighborsClassifier(n_neighbors=10)

In [167]:
clf.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                     weights='uniform')

In [168]:
y_pred = clf.predict(X_test)

In [169]:
accuracy = np.sum(y_pred == y_test)/y_pred.shape[0]

In [170]:
accuracy

0.6910994764397905

In [171]:
accuracy = 0
y_pr = 0
for number_neighbors in range(2,12):
    clf = KNeighborsClassifier(n_neighbors=number_neighbors)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    y_proba = clf.predict_proba(X_test)
    new_acc = np.sum(y_pred == y_test)/y_pred.shape[0]
    if new_acc > accuracy:
        accuracy = np.sum(y_pred == y_test)/y_pred.shape[0]
        y_pr = y_proba
        y_pred1 = y_pred

In [172]:
accuracy

0.7041884816753927

In [173]:
# uzyskac model z accuracy 0.9 (przygotowac inne metryki)

In [174]:
df[df['name'] == '2BFC3C2F3697457']

,offer_class,accepted,name,gender,age,phone_calls,emails,customer_code,salary,offer_code,customer_type,number,offer_value,estimated_expenses,center,difference_between_salary_and_estimated_expenses
34,Medium,no,2BFC3C2F3697457,male,42.0,0.0,1.0,110489,2655.0,11,S,90F8B,NaN,7051.0,A,-4396.0


In [175]:
X[X.index == '2BFC3C2F3697457']

,age,phone_calls,emails,salary,offer_value,estimated_expenses,is_male,medium_offer,premium_offer
name,,,,,,,,,
2BFC3C2F3697457,42.0,0.0,1.0,2655.0,128.693732,7051.0,True,True,False


In [176]:
X['age'].nunique()

72

In [177]:
df[df['name'] == '94A7306C7A23470']

,offer_class,accepted,name,gender,age,phone_calls,emails,customer_code,salary,offer_code,customer_type,number,offer_value,estimated_expenses,center,difference_between_salary_and_estimated_expenses
150,Medium,yes,94A7306C7A23470,male,27.0,0.0,1.0,PC 17572,7672.92,B07,C,DACCC,NaN,2298.0,B,5374.92


In [178]:
#X

In [179]:
# feature engineering

In [180]:
y_pred1

array([False, False,  True, False, False,  True, False,  True, False,
       False, False,  True, False, False, False, False, False, False,
        True,  True, False, False, False, False, False,  True, False,
       False, False, False, False, False, False, False, False, False,
        True, False, False, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True,  True,  True, False, False,
       False, False, False, False, False, False, False, False,  True,
       False, False,  True, False,  True,  True, False, False, False,
       False,  True, False,  True, False,  True, False,  True, False,
       False, False, False, False, False, False, False, False, False,
       False, False,  True, False, False, False, False, False, False,
        True, False, False,  True, False, False, False,  True, False,
        True, False, False, False, False, False, False, False,  True,
       False, False,

In [181]:
# y_pr

In [182]:
from sklearn.linear_model import LogisticRegression

In [183]:
clf = LogisticRegression(penalty='elasticnet', solver='saga', random_state=1337, l1_ratio=0.5, max_iter=100000)

In [184]:
clf.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=0.5, max_iter=100000,
                   multi_class='warn', n_jobs=None, penalty='elasticnet',
                   random_state=1337, solver='saga', tol=0.0001, verbose=0,
                   warm_start=False)

In [185]:
y_test = y_test.to_numpy()

In [186]:
# y_test = y_test.reshape(1, -1)

In [187]:
clf.predict(y_test)

ValueError: Expected 2D array, got 1D array instead:
array=[ True False  True False  True  True  True False False False False False
 False  True  True False False False  True False False  True False False
 False False False  True False  True  True False False False False False
  True False False False False False  True False False False False False
 False False False False False False False False False False  True False
  True False False False  True False False  True  True False  True False
 False  True False  True False  True  True False False  True  True False
  True  True  True False  True  True False False  True False  True False
 False  True False  True  True  True  True  True False False  True False
 False  True False False  True False False False False  True False False
 False False  True False  True  True False  True  True  True False False
  True False False  True  True False False  True  True False False  True
 False  True False False False False False False  True False  Tru

In [188]:
X.head()

,age,phone_calls,emails,salary,offer_value,estimated_expenses,is_male,medium_offer,premium_offer
name,,,,,,,,,
C7CBB5C5613449B,29.000000,0.0,0.0,21133.75,57.426571,4692.0,False,True,False
CFD09C0248BB417,33.511973,1.0,2.0,15155.00,141.639912,3164.0,True,True,False
A2A0DC541977473,33.511973,1.0,3.0,15155.00,154.821130,1852.0,False,True,False
9068458EB70D427,30.000000,1.0,2.0,15155.00,106.256196,3753.0,True,True,False
46F0CD19CF71429,25.000000,1.0,2.0,15155.00,139.237147,2410.0,False,True,False


In [189]:
df['accepted'].value_counts()

no     800
yes    489
Name: accepted, dtype: int64

In [190]:
df['offer_class'].value_counts()

Premium    695
Medium     322
High       272
Name: offer_class, dtype: int64

In [191]:
X.head()

,age,phone_calls,emails,salary,offer_value,estimated_expenses,is_male,medium_offer,premium_offer
name,,,,,,,,,
C7CBB5C5613449B,29.000000,0.0,0.0,21133.75,57.426571,4692.0,False,True,False
CFD09C0248BB417,33.511973,1.0,2.0,15155.00,141.639912,3164.0,True,True,False
A2A0DC541977473,33.511973,1.0,3.0,15155.00,154.821130,1852.0,False,True,False
9068458EB70D427,30.000000,1.0,2.0,15155.00,106.256196,3753.0,True,True,False
46F0CD19CF71429,25.000000,1.0,2.0,15155.00,139.237147,2410.0,False,True,False


In [192]:
X['offer_value'].min()

50.02261933

In [193]:
X['offer_value'].max()

368.6685336

In [194]:
X['offer_value'].quantile(0.95)

240.57297111999992

In [195]:
X['offer_value'].quantile(0.05) #do some boxplots/ #outliers are removed from training dataset

56.450273038000006

In [196]:
X['offer_value']

name
C7CBB5C5613449B     57.426571
CFD09C0248BB417    141.639912
A2A0DC541977473    154.821130
9068458EB70D427    106.256196
46F0CD19CF71429    139.237147
060A000A1260427     62.169773
A472C30155B64A4     60.742667
27C908A4720E4AD    289.613239
F1DB146F2D264B9     75.369051
9E8DFA1C6F494E6    102.144904
F36F20E6C4C9460    128.277347
B6409DE9C35C4BB    161.352318
1B1B5AFE5EF249A     87.760293
BC61E9C5A9B14B1     58.312947
AA6AFCF561E940D     87.717730
1080E57315CC4A1    154.593613
E21190622F1B423    172.304627
F1E0937D8B88439    203.472922
CD794AE9A0794AE    162.987764
FB7DB69F3E0F4BF     68.472646
AD2D8614D1CF402     85.559777
755DBCB134CD4A4    142.918433
95DC85EEFAA9432    110.021422
955E62E905E845B    212.229544
30F9A572545B4CB     52.216582
17DB1678C2DE4F2     91.779684
5FAC6586BAFF41F    181.884562
EA33697095194F2    276.642656
6D0276C6987E4D1     93.617746
3BAC523AC0E0418    113.493852
D42E80D2B18941F    143.010657
8DD730EF690C451     88.351355
5AC787A4EB184B3     77.658872
E1451

notes on balancing the dataset: 

Nice kernel! For oversampling, I've found that SMOTE combined with edited nearest neighbors (ENN) tends to perform slightly better than SMOTE, but takes longer to run. It's definitely worth looking into. Here's the code for that: https://imbalanced-learn.readthedocs.io/en/stable/generated/imblearn.combine.SMOTEENN.html



In [197]:
df[df['age'] > 50]['accepted'].value_counts()

no     56
yes    38
Name: accepted, dtype: int64

In [198]:
df[df['age'] < 20]['accepted'].value_counts()

no     45
yes    23
Name: accepted, dtype: int64

In [199]:
X.shape

(1273, 9)

In [200]:
X.head()

,age,phone_calls,emails,salary,offer_value,estimated_expenses,is_male,medium_offer,premium_offer
name,,,,,,,,,
C7CBB5C5613449B,29.000000,0.0,0.0,21133.75,57.426571,4692.0,False,True,False
CFD09C0248BB417,33.511973,1.0,2.0,15155.00,141.639912,3164.0,True,True,False
A2A0DC541977473,33.511973,1.0,3.0,15155.00,154.821130,1852.0,False,True,False
9068458EB70D427,30.000000,1.0,2.0,15155.00,106.256196,3753.0,True,True,False
46F0CD19CF71429,25.000000,1.0,2.0,15155.00,139.237147,2410.0,False,True,False


In [201]:
y.head()

name
C7CBB5C5613449B     True
CFD09C0248BB417     True
A2A0DC541977473    False
9068458EB70D427    False
46F0CD19CF71429    False
Name: target, dtype: bool

In [202]:
merged = pd.merge(X, y, on='name')

In [203]:
merged.head()

,age,phone_calls,emails,salary,offer_value,estimated_expenses,is_male,medium_offer,premium_offer,target
name,,,,,,,,,,
C7CBB5C5613449B,29.000000,0.0,0.0,21133.75,57.426571,4692.0,False,True,False,True
CFD09C0248BB417,33.511973,1.0,2.0,15155.00,141.639912,3164.0,True,True,False,True
A2A0DC541977473,33.511973,1.0,3.0,15155.00,154.821130,1852.0,False,True,False,False
9068458EB70D427,30.000000,1.0,2.0,15155.00,106.256196,3753.0,True,True,False,False
46F0CD19CF71429,25.000000,1.0,2.0,15155.00,139.237147,2410.0,False,True,False,False


In [204]:
merged[merged['salary'] < 1500]['target'].value_counts()

False    495
True     164
Name: target, dtype: int64

In [205]:
merged[merged['salary'] > 5000]['target'].value_counts()

True     151
False     80
Name: target, dtype: int64

In [206]:
merged[['target', 'salary', 'age']].groupby(['age', 'target']).count().sort_values('age')

salary
age       target        
18.000000 False       25
          True        13
18.500000 False        3
19.000000 False       17
          True        10
20.000000 False       14
          True         8
20.500000 False        1
21.000000 False       30
          True        11
22.000000 False       22
          True        19
22.500000 False        1
23.000000 True        10
          False       16
23.500000 False        1
24.000000 False       25
          True        20
24.500000 False        1
25.000000 False       23
          True        11
26.000000 False       19
          True        11
26.500000 False        1
27.000000 False       17
          True        13
28.000000 False       24
          True         8
28.500000 False        3
29.000000 True        12
          False       16
30.000000 False       25
          True        15
30.500000 False        2
31.000000 False       11
          True        12
32.000000 False       12
          True         9
32.500000 False        3
          True         1
33.000000 False       12
          True         9
33.511973 False      257
          True       150
34.000000 True         6
          False       10
34.500000 False        2
35.000000 False        9
          True        11
36.000000 False       17
          True        13
36.500000 False        1
          True         1
37.000000 False        7
          True         2
38.000000 False        8
          True         6
38.500000 False        1
39.000000 True         8
          False       12
40.000000 False       11
          True         5
40.500000 False        3
41.000000 False        9
          True         2
42.000000 False       12
          True         6
43.000000 False        5
          True         3
44.000000 False        7
          True         3
45.000000 False        7
          True        12
45.500000 False        2
46.000000 False        6
47.000000 False       11
          True         3
48.000000 False        4
          True        10
49.000000 False        4
          True         5
50.000000 False        9
          True         5
51.000000 False        5
          True         3
52.000000 False        3
          True         3
53.000000 True         4
54.000000 True         5
          False        5
55.000000 False        4
          True         4
55.500000 False        1
56.000000 False        2
          True         2
57.000000 False        5
58.000000 False        2
          True         3
59.000000 False        2
          True         1
60.000000 False        2
          True         4
61.000000 False        5
62.000000 True         2
          False        3
63.000000 False        2
          True         2
64.000000 False        3
          True         2
65.000000 False        3
66.000000 False        1
67.000000 False        1
70.000000 False        2
70.500000 False        1
71.000000 False        2
74.000000 False        1
76.000000 True         1
80.000000 True         1

In [207]:
merged[['target', 'salary', 'age']].groupby(['salary', 'target']).count().sort_values('salary')

age
salary   target     
0.00     False    15
         True      2
317.08   True      1
401.25   False     1
500.00   False     1
623.75   False     1
643.75   False     2
645.00   False     1
649.58   False     3
675.00   False     2
685.83   False     1
695.00   False     1
         True      1
697.50   False     1
         True      1
700.00   False     1
         True      1
704.58   False     1
705.00   False     8
         True      1
705.42   False     2
712.50   False     4
714.17   True      1
722.50   False    16
         True      5
722.92   False    17
         True      7
725.00   False    17
728.33   False     1
731.25   False     1
749.58   False     2
         True      1
752.08   False     1
755.00   True      2
         False     6
757.50   False     1
757.92   True      1
762.92   False     2
765.00   False     3
         True      3
772.08   True      1
772.50   False     1
         True      1
772.92   False     1
773.33   False     4
         True      3
773.75   True      1
         False     1
774.17   False     1
775.00   False    38
         True     15
777.50   False    19
         True      7
777.92   True      1
778.75   True      1
779.58   False     5
         True      5
780.00   False     1
782.08   True      1
782.92   True      1
         False     2
785.00   False     1
785.42   False    16
         True      4
787.50   False     1
787.92   False     6
         True      3
788.75   False     1
         True      1
789.58   False    46
         True      1
792.50   True      9
         False    14
802.92   True      1
805.00   False    49
         True      8
811.25   False     1
         True      1
813.75   False     1
815.83   False     1
830.00   False     1
836.25   False     1
840.42   False     1
843.33   False     1
851.67   True      2
865.42   False     1
866.25   False    20
         True      1
868.33   True      1
871.25   False     2
885.00   False     1
896.25   True      1
900.00   False     2
921.67   False     1
922.50   False     2
         True      1
932.50   False     1
935.00   False     1
         True      1
947.50   False     1
948.33   False     1
950.00   False     9
         True      3
958.75   False     1
         True      1
968.75   False     1
982.50   False     2
983.75   False     1
984.17   True      1
984.58   False     1
1017.08  False     1
1046.25  False     2
1050.00  False    23
         True     12
1051.67  False     1
1070.83  False     1
1113.33  True      3
1124.17  True      2
1150.00  False     6
1200.00  True      1
1218.33  False     2
1227.50  False     1
1228.75  True      2
1235.00  False     3
         True      2
1247.50  True      4
1252.50  False     1
1265.00  True      1
1273.75  True      1
1287.50  False     2
1300.00  False    42
         True     17
1341.67  True      2
1350.00  False     5
         True      2
1377.50  False     3
1379.17  True      1
1385.83  True      3
1386.25  True      2
1390.00  False     1
         True      1
1400.00  False     1
1410.83  False     1
         True      1
1440.00  False     3
1445.42  False     9
         True      1
1445.83  False     4
1450.00  False     9
         True      2
1500.00  False     1
1503.33  False     1
1504.58  False     3
1505.00  False     1
1510.00  False     2
1524.58  True      6
         False     3
1550.00  False     5
         True      4
1555.00  False     2
1557.92  False     1
1574.17  True      3
1575.00  True      2
1585.00  False     1
         True      2
1590.00  True      3
1600.00  True      1
         False     1
1610.00  True      3
         False     9
1670.00  True      3
1740.00  True      2
1780.00  False     2
1800.00  False     4
1875.00  True      3
1878.75  False     1
         True      1
1925.83  True      4
1950.00  True      2
1996.67  False     2
2021.25  False     3
2025.00  True      1
         False     2
2052.50  False     1
         True      1
2057.50  False     1
         True      3
2100.00  False     9
         True      5
2107.5

In [208]:
df['number'][:5]

0       9E9FA
1       1E53D
2    1.36E+06
3       F6529
4       E2FDF
Name: number, dtype: object

In [209]:
df.columns

Index(['offer_class', 'accepted', 'name', 'gender', 'age', 'phone_calls',
       'emails', 'customer_code', 'salary', 'offer_code', 'customer_type',
       'number', 'offer_value', 'estimated_expenses', 'center',
       'difference_between_salary_and_estimated_expenses'],
      dtype='object')

In [210]:
df[df['difference_between_salary_and_estimated_expenses']>0]['accepted'].value_counts()

yes    159
no     119
Name: accepted, dtype: int64

In [211]:
df[df['difference_between_salary_and_estimated_expenses']<0]['accepted'].value_counts()

no     678
yes    322
Name: accepted, dtype: int64

In [212]:
df.columns

Index(['offer_class', 'accepted', 'name', 'gender', 'age', 'phone_calls',
       'emails', 'customer_code', 'salary', 'offer_code', 'customer_type',
       'number', 'offer_value', 'estimated_expenses', 'center',
       'difference_between_salary_and_estimated_expenses'],
      dtype='object')

In [213]:
# czy to, ze emaile sa w ciagu 6 miesiecy a polaczenia w 3, sprawia roznice? przeskalowac jakos dane do jednego okresu?